In [ ]:
import os
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_STATE = 42

In [ ]:
features = ['sex', 'age', 'failures', 'higher', 'paid', 'absences', 'G_Avg']
target = 'G3_10'

In [ ]:
# --- 2) Các file kết quả khả dĩ (tạo bởi file training) ---
possible_results = [
'results_df.pkl',
'models_results.pkl',
'baseline_results.pkl',
'candidates_results.pkl',
'combined_results.pkl'
]


results_df = None
loaded_from = None


# thử load bảng kết quả (DataFrame hoặc list of dict)
for p in possible_results:
if os.path.exists(p):
try:
with open(p, 'rb') as f:
loaded = pickle.load(f)
if isinstance(loaded, pd.DataFrame):
results_df = loaded.copy()
else:
results_df = pd.DataFrame(loaded)
loaded_from = p
print(f"Loaded results from {p}")
break
except Exception as e:
print(f"Found {p} but failed to load: {e}")

In [ ]:
# --- 3) Nếu không có results_df, cố gắng load best_pipeline_model.pkl ---
eval_context = None
best_model_path = 'best_pipeline_model.pkl'
if results_df is None:
if os.path.exists(best_model_path):
# cần DataFrame 'df' để tái tạo X_test/y_test giống training
if 'df' not in globals():
# thử load df từ 1 số tên file thông dụng
tried = False
for cand in ['df.pkl', 'data.pkl', 'data.csv', 'student.csv', 'students.csv']:
if os.path.exists(cand):
tried = True
try:
if cand.endswith('.pkl'):
df = pd.read_pickle(cand)
else:
df = pd.read_csv(cand)
print(f"Loaded data from {cand}")
break
except Exception as e:
print(f"Found {cand} but failed to read: {e}")
if not tried:
# không tìm được file df, raise error rõ ràng
raise RuntimeError("Không tìm thấy 'results_df' và biến 'df' chưa được load.\n"
"Hãy load DataFrame (ví dụ df = pd.read_csv(...)) trước khi chạy evaluation, "
"hoặc lưu results_df trong file training.")
# tái tạo split giống training
X = df[features].copy()
y = df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=RANDOM_STATE
)
# load best pipeline
best_pipe = joblib.load(best_model_path)
y_pred = best_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
results_df = pd.DataFrame([{
'model': 'best_pipeline_model',
'best_params': {},
'rmse': rmse,
'r2': r2
}])
eval_context = {'X_test': X_test, 'y_test': y_test, 'y_pred': y_pred}
loaded_from = best_model_path
print(f"Built results_df from {best_model_path}")
else:
raise FileNotFoundError("Không tìm thấy file results của training hoặc best_pipeline_model.pkl. Chạy training và lưu model trước khi chạy evaluation.")

In [ ]:
# --- 4) Chuẩn hoá results_df nếu cần (nếu chứa 'pipeline', compute metrics) ---
if 'rmse' not in results_df.columns or 'r2' not in results_df.columns:
if 'pipeline' in results_df.columns:
if 'df' not in globals():
raise RuntimeError("results_df chứa pipeline nhưng biến 'df' chưa được load. Load df để đánh giá.")
X = df[features].copy()
y = df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=RANDOM_STATE
)
rows = []
for r in results_df.to_dict(orient='records'):
pipe = r.get('pipeline')
if pipe is None:
continue
yp = pipe.predict(X_test)
rows.append({
'model': r.get('model', 'unknown'),
'best_params': r.get('best_params', {}),
'rmse': np.sqrt(mean_squared_error(y_test, yp)),
'r2': r2_score(y_test, yp)
})
results_df = pd.DataFrame(rows)
eval_context = {'X_test': X_test, 'y_test': y_test}
else:
raise RuntimeError('File kết quả load được nhưng thiếu cột rmse/r2 và không có pipeline để tính.')


# ensure model column
if 'model' not in results_df.columns:
results_df['model'] = results_df.index.astype(str)

In [ ]:
# --- 5) Hiển thị bảng so sánh và lưu csv ---
results_df = results_df.sort_values('rmse').reset_index(drop=True)
print(f"\n--- Model comparison (sorted by RMSE) — source: {loaded_from} ---")
print(results_df[['model', 'best_params', 'rmse', 'r2']])
results_df.to_csv('models_comparison_summary.csv', index=False)
print('\nSaved summary to models_comparison_summary.csv')

In [ ]:
# --- 6) VẼ biểu đồ RMSE và R2 ---
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.bar(results_df['model'], results_df['rmse'])
plt.xticks(rotation=45, ha='right')
plt.title('RMSE by model')


plt.subplot(1,2,2)
plt.bar(results_df['model'], results_df['r2'])
plt.xticks(rotation=45, ha='right')
plt.title('R2 by model')


plt.tight_layout()
plt.show()

In [ ]:
if eval_context is not None:
X_test = eval_context.get('X_test')
y_test = eval_context.get('y_test')
y_pred = eval_context.get('y_pred')
if y_pred is None and os.path.exists(best_model_path):
best_pipe = joblib.load(best_model_path)
y_pred = best_pipe.predict(X_test)
if y_pred is not None:
plt.figure(figsize=(6,5))
plt.scatter(y_test, y_pred, alpha=0.6)
mn = min(min(y_test), min(y_pred))
mx = max(max(y_test), max(y_pred))
plt.plot([mn, mx], [mn, mx], linestyle='--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted (best_pipeline_model)')
plt.tight_layout()
plt.show()